In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from PIL import Image
import os
import cv2

def dataset_func(diretorio):
    imagens = []
    for filename in os.listdir(diretorio):
        caminho = os.path.join(diretorio, filename)
        if os.path.isfile(caminho):
            input_image = cv2.imread(caminho)
            if input_image is not None:
                imagens.append(input_image)
            else:
                print(f"Erro ao ler imagem: {caminho}")
    return imagens

input_image_path = "/content/drive/MyDrive/u-net_test/img"
input_image = dataset_func(input_image_path)


def masks_func(diretorio):
    imagens = []
    for filename in os.listdir(diretorio):
        caminho = os.path.join(diretorio, filename)
        if os.path.isfile(caminho):
            input_mask = cv2.imread(caminho, cv2.IMREAD_GRAYSCALE)  # Lendo a imagem em escala de cinza
            if input_mask is not None:
                # Aplicando thresholding
                _, thresholded_mask = cv2.threshold(input_mask, 127, 255, cv2.THRESH_BINARY)
                imagens.append(thresholded_mask)
            else:
                print(f"Erro ao ler imagem: {caminho}")
    return imagens

masks_path = "/content/drive/MyDrive/u-net_test/masks"
input_mask = masks_func(masks_path)

In [ ]:
def resize(input_image, input_masks):
    # Adicionando um eixo para os canais de cor, se necessário
    if input_image.ndim == 2:
        input_image = np.expand_dims(input_image, axis=-1)
    if input_masks.ndim == 2:
        input_masks = np.expand_dims(input_masks, axis=-1)

    input_image = tf.image.resize(input_image, (128, 128), method="nearest")
    input_masks = tf.image.resize(input_masks, (128, 128), method="nearest")

    return input_image, input_masks

def augment(input_image, input_mask):
   if tf.random.uniform(()) > 0.5:
       # Random flipping of the image and mask
       input_image = tf.image.flip_left_right(input_image)
       input_mask = tf.image.flip_left_right(input_mask)
   return input_image, input_mask

def normalize(input_image, input_mask):
   input_image = tf.cast(input_image, tf.float32) / 255.0
   input_mask -= 1
   return input_image, input_mask

def load_image_train(images, masks):
    resized_images = []
    resized_masks = []
    for i in range(len(images)):
        input_image = images[i]
        input_mask = masks[i]
        input_image, input_mask = resize(input_image, input_mask)
        input_image, input_mask = augment(input_image, input_mask)
        input_image, input_mask = normalize(input_image, input_mask)
        resized_images.append(input_image)
        resized_masks.append(input_mask)
    return resized_images, resized_masks

def load_image_test(input_image, input_mask):
   input_image, input_mask = resize(input_image, input_mask)
   input_image, input_mask = normalize(input_image, input_mask)
   return input_image, input_mask

input_image = dataset_func(input_image_path)
input_mask = masks_func(masks_path)
input_image, input_mask = load_image_train(input_image, input_mask)



In [ ]:
# Dividindo os dados em conjuntos de treinamento e teste
train_size = int(0.8 * len(input_image))
train_image, test_image = input_image[:train_size], input_image[train_size:]
train_mask, test_mask = input_mask[:train_size], input_mask[train_size:]

# Aplicando a função resize aos conjuntos de treinamento e teste
train_resized = [resize(image, mask) for image, mask in zip(train_image, train_mask)]
test_resized = [resize(image, mask) for image, mask in zip(test_image, test_mask)]

# Convertendo para tensores TensorFlow
train_resized_image = tf.convert_to_tensor([image for image, _ in train_resized])
train_resized_mask = tf.convert_to_tensor([mask for _, mask in train_resized])
test_resized_image = tf.convert_to_tensor([image for image, _ in test_resized])
test_resized_mask = tf.convert_to_tensor([mask for _, mask in test_resized])

# Criando tf.data.Dataset a partir dos conjuntos de dados redimensionados
train_dataset = tf.data.Dataset.from_tensor_slices((train_resized_image, train_resized_mask))
test_dataset = tf.data.Dataset.from_tensor_slices((test_resized_image, test_resized_mask))



In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
train_batches = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_batches = train_batches.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
validation_batches = test_dataset.take(3000).batch(BATCH_SIZE)
test_batches = test_dataset.skip(3000).take(669).batch(BATCH_SIZE)

In [ ]:
def display(display_list):
 plt.figure(figsize=(15, 15))
 title = ["Input Image", "True Mask", "Predicted Mask"]
 for i in range(len(display_list)):
   plt.subplot(1, len(display_list), i+1)
   plt.title(title[i])
   plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
   plt.axis("off")
 plt.show()
sample_batch = next(iter(train_batches))
random_index = np.random.choice(sample_batch[0].shape[0])
sample_image, sample_mask = sample_batch[0][random_index], sample_batch[1][random_index]
display([sample_image, sample_mask])

In [ ]:
def double_conv_block(x, n_filters):
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal")(x)
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal")(x)
    return x

In [ ]:
def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p

In [ ]:
def upsample_block(x, conv_features, n_filters):
    # upsample
    x = layers.Conv2DTranspose(n_filters, 3, strides=2, padding="same")(x)
    # concatenate
    x = layers.concatenate([x, conv_features])
    # dropout
    x = layers.Dropout(0.3)(x)
    # Conv2D twice with ReLU activation
    x = double_conv_block(x, n_filters)
    return x


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def build_unet_model():
    # inputs
    inputs = layers.Input(shape=(128, 128, 3))

    # encoder: contracting path - downsample
    # 1 - downsample
    f1, p1 = downsample_block(inputs, 64)
    # 2 - downsample
    f2, p2 = downsample_block(p1, 128)
    # 3 - downsample
    f3, p3 = downsample_block(p2, 256)
    # 4 - downsample
    f4, p4 = downsample_block(p3, 512)

    # 5 - bottleneck
    bottleneck = double_conv_block(p4, 1024)

    # decoder: expanding path - upsample
    # 6 - upsample
    u6 = upsample_block(bottleneck, f4, 512)
    # 7 - upsample
    u7 = upsample_block(u6, f3, 256)
    # 8 - upsample
    u8 = upsample_block(u7, f2, 128)
    # 9 - upsample
    u9 = upsample_block(u8, f1, 64)

    # outputs
    activation = "sigmoid"  # for binary segmentation
    outputs = layers.Conv2D(1, 1, padding="same", activation=activation)(u9)

    # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model


In [ ]:
unet_model = build_unet_model()

In [ ]:
unet_model.compile(optimizer=tf.keras.optimizers.Adam(),
                   loss="binary_crossentropy",
                   metrics="accuracy")

In [ ]:
# Obtendo o comprimento dos conjuntos de dados input_image e input_mask
TRAIN_LENGTH = len(train_dataset) * 2
TEST_LENGTH = len(test_dataset) * 2

NUM_EPOCHS = 10
VAL_SUBSPLITS = 5
BATCH_SIZE = 10

# Calculando o número de etapas por época e etapas de validação
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE
VALIDATION_STEPS = TEST_LENGTH // BATCH_SIZE

# Verificando se STEPS_PER_EPOCH é zero
if STEPS_PER_EPOCH == 10:
    raise ValueError("O valor de STEPS_PER_EPOCH é zero. Verifique os tamanhos do conjunto de dados e do lote.")

# Agora você pode prosseguir com o treinamento do modelo usando esses valores
model_history = unet_model.fit(train_batches,
                               epochs=NUM_EPOCHS,
                               steps_per_epoch=STEPS_PER_EPOCH,
                               validation_steps=VALIDATION_STEPS,
                               validation_data=test_batches)



In [ ]:
def create_mask(pred_mask):
 pred_mask = tf.argmax(pred_mask, axis=-1)
 pred_mask = pred_mask[..., tf.newaxis]
 return pred_mask[0]
def show_predictions(dataset=None, num=1):
 if dataset:
   for image, mask in dataset.take(num):
     pred_mask = unet_model.predict(image)
     display([image[0], mask[0], create_mask(pred_mask)])
 else:
   display([sample_image, sample_mask,
            create_mask(model.predict(sample_image[tf.newaxis, ...]))])
count = 0
for i in test_batches:
   count +=1
print("number of batches:", count)